# Combine Model

In [40]:
model_transfer = InceptionV3(weights='imagenet', include_top=False, input_shape=(299, 299, 3))

In [46]:
# define combined model
# !!! keep the top layers consistent with the custom model
x = model_transfer.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation="relu")(x)
x = Dropout(0.8)(x)
x = Dense(10, activation="softmax")(x)
model_combined = Model(model_transfer.input, x)

In [49]:
# load custom layer weights
model.load_weights('model_4.h5')  # update this with the most up-to-date weights, 'model' is the custom model

# get weights for inception v3 and for custom layers
inceptionv3_weights = model_transfer.get_weights()
top_layer_weights = model.get_weights()

# combine weights
combined_weights = inceptionv3_weights + top_layer_weights

# set weights for combined model
model_combined.set_weights(combined_weights)

In [57]:
print(len(inceptionv3_weights))
print(len(top_layer_weights))
print(len(combined_weights))

376
4
380


### Continue training combined model

In [62]:
model_combined.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy', metrics = ['accuracy'])

In [65]:
def load_image(path):
    img = Image.open(path)
    img = img.resize(size=(299,299))
    img = np.array(img)
    img = img / 255.0
    # Convert 2-dim gray-scale array to 3-dim RGB array.
    # if (len(img.shape) == 2):
    #     img = np.repeat(img[:, :, np.newaxis], 3, axis=2)
    return img

In [66]:
def generator(X_data, y_data, batch_size):
    i = 0
    while True:
        X_batch = []
        y_batch = []
        
        for b in range(batch_size):
            if i >= len(X_data):
                i = 0
                
            X = X_data[i]
            y = y_data[i]
            X = load_image(X)
            X_batch.append(X)
            y_batch.append(y)
            i += 1
            
        yield np.array(X_batch), np.array(y_batch)

In [63]:
class_dist = dict(image_df_annot.groupby('category_id_model').image_id.count().sort_values())
total = sum(class_dist.values())
class_dist_inverse = {k: total/v for k, v in class_dist.items()}

In [64]:
class_dist_inverse

{6: 128.48837209302326,
 0: 56.905430711610485,
 1: 54.851083032490976,
 3: 25.533032244512132,
 5: 22.13823877606775,
 4: 19.940940037732755,
 2: 19.692183070068854,
 7: 14.614644703619094,
 8: 5.391798079268969,
 9: 1.9332776651159091}

In [68]:
# test on 1024 images
sample_x_train = img_vector_train[:1024]
sample_y_train = cat_id_vector_train[:1024]
sample_x_val = img_vector_val[:64]
sample_y_val = cat_id_vector_val[:64]

In [ ]:
# without reducing empty class, 10 class with adjusted class weights
# train full model incl. inception v3 layers

X_train = sample_x_train
y_train = sample_y_train
X_val = sample_x_val
y_val = sample_y_val

# X_train = img_vector_train
# y_train = cat_id_vector_train
# X_val = img_vector_val
# y_val = cat_id_vector_val

EPOCH = 2
BATCH_SIZE = 64
TRAIN_STEPS = len(X_train)//BATCH_SIZE
VAL_STEPS = len(y_train)//BATCH_SIZE

train_generator = generator(X_train, y_train, BATCH_SIZE)
val_generator = generator(X_val, y_val, BATCH_SIZE)

for i in range(EPOCH):
    model_combined.fit_generator(generator=train_generator, \
                        validation_data=val_generator,\
                        validation_steps=VAL_STEPS, \
                        steps_per_epoch=TRAIN_STEPS, \
                        epochs=1, verbose=1,\
                        class_weight=class_dist_inverse)
                        #use_multiprocessing=True)
    model_combined.save('./weights/combined/model_' + str(i) + '.h5')

Instructions for updating:
Please use Model.fit, which supports generators.
  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 16 steps, validate for 16 steps
 3/16 [====>.........................] - ETA: 5:10 - loss: 27.1582 - accuracy: 0.3385